# Section 3: The Pangeo Approach - An Implementation Guide

## Cloud computing

## Containers

## orchestration  - kubernetes

## distribution - Dask

## Interactivity & portability - Jupyter Labs

## Dashboarding